In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.io import loadmat
import heartpy as hp

In [2]:
df = pd.DataFrame()
indexes = ['Arousal', 'Valence', 'Engagement', 'Liking', 'Familiarity']
dfLabels = pd.DataFrame(index=indexes)

emotionsRaw = loadmat('data/ASCERTAIN/Dt_SelfReports.mat')

for i in range(1, 59):
    print('\r' + str(i) + '/58', end=' ')
    if(len(str(i)) == 1):
        subjNum = "0"+str(i)
    else:
        subjNum = str(i)
    
    path = "data/ASCERTAIN/ECGData/Movie_P" + subjNum
    
    for j in range(1,37):
        clipNum = str(j)
        
        filePath = "/ECG_Clip" + clipNum + ".mat"
        fileData = loadmat(path + filePath)
        colName = "P" + subjNum + "C" + clipNum

        if(i < 9):
            left = fileData['Data_ECG'][:,2]
            #right = ecg['Data_ECG'][:,1]
        else:
            left = fileData['Data_ECG'][:,5]
            #right = ecg['Data_ECG'][:,4]

        #Emotions
        emotList = []
        for r in range(0,5):
            emotList.append(emotionsRaw['Ratings'][r][i-1][j-1])

        df = pd.concat([df, pd.DataFrame({colName : left})], axis=1)
        dfLabels = pd.concat([dfLabels, pd.DataFrame({colName : emotList}, index=indexes)], axis=1)

58/58 

In [3]:
print(dfLabels.shape)
dfLabels.head()

(5, 2088)


,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
Arousal,1.0,3.0,1.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,...,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
Valence,1.0,2.0,-2.0,0.0,1.0,0.0,1.0,2.0,2.0,3.0,...,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-3.0,-2.0,-2.0
Engagement,4.0,5.0,3.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0
Liking,5.0,4.0,3.0,3.0,4.0,3.0,4.0,5.0,5.0,6.0,...,1.0,1.0,1.0,2.0,2.0,1.0,2.0,0.0,1.0,1.0
Familiarity,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,4.0,...,1.0,0.0,0.0,0.0,3.0,2.0,1.0,0.0,2.0,3.0


In [4]:
print(df.shape)
df.head() 

(39923, 2088)


,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
0,1984.0,2056.0,1833.0,2070.0,2063.0,2049.0,2034.0,2052.0,2035.0,2019.0,...,2032.0,2004.0,2031.0,2048.0,2073.0,2047.0,2054.0,2039.0,2094.0,2019.0
1,1982.0,2056.0,1832.0,2071.0,2064.0,2040.0,2033.0,2040.0,2036.0,2026.0,...,2055.0,1990.0,2018.0,2066.0,2096.0,2055.0,2057.0,2049.0,2084.0,2029.0
2,1979.0,2054.0,1844.0,2073.0,2070.0,2039.0,2041.0,2049.0,2035.0,2015.0,...,2039.0,1999.0,2019.0,2076.0,2099.0,2047.0,2033.0,2044.0,2062.0,2026.0
3,1979.0,2053.0,1842.0,2067.0,2078.0,2043.0,2030.0,2058.0,2035.0,2026.0,...,2028.0,2021.0,2029.0,2070.0,2099.0,2066.0,2059.0,2039.0,2065.0,2001.0
4,1981.0,2069.0,1861.0,2074.0,2065.0,2037.0,2036.0,2038.0,2036.0,2014.0,...,2043.0,2039.0,2038.0,2056.0,2056.0,2050.0,2073.0,2060.0,2060.0,1998.0


In [5]:
df.to_csv('data/ASCERTAIN_ECG_Raw.csv', index=False)
dfLabels.to_csv('data/ASCERTAIN_Emotions_Raw.csv')

In [64]:
dfRaw = pd.read_csv('data/ASCERTAIN_ECG_Raw.csv')
dfLabels = pd.read_csv('data/ASCERTAIN_Emotions_Raw.csv')

In [33]:
def calculateBPM(seriesIn, colName):
#Calculate moving average with 0.75s in both directions, then append to dataset
    print(colName)
    hrw = 0.75 #One-sided window size, as proportion of the sampling frequency
    fs = 256 #The example dataset was recorded at 256Hz
    
    series = seriesIn.copy()
    print(series.shape)
    dropIdx = len(series.dropna())
    series = series.truncate(after=dropIdx)
    print(series.shape)

    minuteHz = 15360
    currIdx = 15360
    bpmList = []
    
#     if(len(series)<minuteHz):
#         print('ERROR0')
#         return []
    
    while currIdx < len(series):
#         print(currIdx)
        currSlice = pd.DataFrame({colName : list(series.iloc[currIdx-minuteHz:currIdx])})
        currIdx += 256
    
        numValues = int(len(currSlice)*0.05)

        peakAvg = currSlice[colName].nlargest(numValues).mean()
        avg = currSlice[colName].mean()

        raiseVal = peakAvg/avg

        mov_avg = currSlice[colName].rolling(int(hrw*fs)).mean() #Calculate moving average
        #Impute where moving average function returns NaN, which is the beginning of the signal where x hrw
        avg_hr = (np.mean(currSlice[colName]))
        mov_avg = [avg_hr if math.isnan(x) else x for x in mov_avg]
        mov_avg = [x*raiseVal for x in mov_avg] #For now we raise the average by 20% to prevent the secondary heart contraction from interfering, in part 2 we will do this dynamically
        currSlice['rollingmean'] = mov_avg #Append the moving average to the dataframe

        #Mark regions of interest
        window = []
        peaklist = []
        listpos = 0 #We use a counter to move over the different data columns
        try:
            for datapoint in currSlice[colName]:
                rollingmean = currSlice.rollingmean[listpos] #Get local mean
                if (datapoint < rollingmean) and (len(window) < 1): #If no detectable R-complex activity -> do nothing
                    listpos += 1
                elif (datapoint > rollingmean): #If signal comes above local mean, mark ROI
                    window.append(datapoint)
                    listpos += 1
                else: #If signal drops below local mean -> determine highest point
                    maximum = max(window)
                    beatposition = listpos - len(window) + (window.index(max(window))) #Notate the position of the point on the X-axis
                    peaklist.append(beatposition) #Add detected peak to list
                    window = [] #Clear marked ROI
                    listpos += 1
        except ValueError:
            print('ERROR1')
            return []

        ybeat = [currSlice[colName][x] for x in peaklist] #Get the y-value of all peaks for plotting purposes

#         plt.title("Detected peaks in signal")
#         plt.plot(currSlice[colName], alpha=0.5, color='blue') #Plot semi-transparent HR
#         plt.plot(mov_avg, color ='green') #Plot moving average
#         plt.scatter(peaklist, ybeat, color='red') #Plot detected peaks
#         plt.show()

        RR_list = []
        cnt = 0
#         print(peaklist[-1])

        while (cnt < (len(peaklist)-1)):
            RR_interval = (peaklist[cnt+1] - peaklist[cnt]) #Calculate distance between beats in # of samples
            ms_dist = ((RR_interval / fs) * 1000.0) #Convert sample distances to ms distances
            RR_list.append(ms_dist) #Append to list
            cnt += 1
            
#         if(str(np.mean(RR_list)) != 'nan'):
        try:
            bpm = round(60000 / np.mean(RR_list)) #60000 ms (1 minute) / average R-R interval of signal
            bpmList.append(bpm)
        except ValueError:
            print('ERROR2')
            return []
    #     print("Average Heart Beat is: %.01f" %bpm) #Round off to 1 decimal and print

    return bpmList

In [34]:
hrDf = pd.DataFrame()

for col in df.columns:
    hrDf = pd.concat([hrDf, pd.DataFrame({col : calculateBPM(df[col], col)})], axis=1)

P01C1
(39923,)
(27947,)
P01C2
(39923,)
(18246,)
P01C3
(39923,)
(15160,)
P01C4
(39923,)
(23251,)
P01C5
(39923,)
(26965,)
P01C6
(39923,)
(14391,)
P01C7
(39923,)
(24830,)
P01C8
(39923,)
(18337,)
P01C9
(39923,)
(19036,)
P01C10
(39923,)
(24766,)
P01C11
(39923,)
(16453,)
P01C12
(39923,)
(24683,)
P01C13
(39923,)
(14138,)
P01C14
(39923,)
(14408,)
P01C15
(39923,)
(14135,)
P01C16
(39923,)
(16476,)
P01C17
(39923,)
(21914,)
P01C18
(39923,)
(20825,)
P01C19
(39923,)
(29075,)
P01C20
(39923,)
(16513,)
P01C21
(39923,)
(23740,)
P01C22
(39923,)
(24250,)
P01C23
(39923,)
(18507,)
P01C24
(39923,)
(24410,)
P01C25
(39923,)
(31634,)
P01C26
(39923,)
(21594,)
P01C27
(39923,)
(19268,)
P01C28
(39923,)
(23689,)
P01C29
(39923,)
(23386,)
P01C30
(39923,)
(21572,)
P01C31
(39923,)
(34367,)
P01C32
(39923,)
(29932,)
P01C33
(39923,)
(32365,)
P01C34
(39923,)
(16457,)
P01C35
(39923,)
(24860,)
P01C36
(39923,)
(16958,)
P02C1
(39923,)
(28048,)
P02C2
(39923,)
(18256,)
ERROR2
P02C3
(39923,)
(15165,)
P02C4
(39923,)
(23176,)
P02C5


P09C35
(39923,)
(24925,)
P09C36
(39923,)
(16966,)
ERROR2
P10C1
(39923,)
(27881,)
P10C2
(39923,)
(18251,)
P10C3
(39923,)
(15165,)
P10C4
(39923,)
(23718,)
P10C5
(39923,)
(26911,)
P10C6
(39923,)
(14406,)
P10C7
(39923,)
(24794,)
P10C8
(39923,)
(18349,)
P10C9
(39923,)
(19036,)
P10C10
(39923,)
(24784,)
P10C11
(39923,)
(16465,)
P10C12
(39923,)
(24693,)
P10C13
(39923,)
(14145,)
P10C14
(39923,)
(14411,)
P10C15
(39923,)
(14158,)
P10C16
(39923,)
(16461,)
P10C17
(39923,)
(21934,)
P10C18
(39923,)
(20820,)
P10C19
(39923,)
(29136,)
P10C20
(39923,)
(16488,)
P10C21
(39923,)
(23674,)
P10C22
(39923,)
(24282,)
P10C23
(39923,)
(18514,)
P10C24
(39923,)
(24339,)
P10C25
(39923,)
(31633,)
P10C26
(39923,)
(21643,)
P10C27
(39923,)
(19282,)
P10C28
(39923,)
(23737,)
P10C29
(39923,)
(23396,)
P10C30
(39923,)
(21592,)
P10C31
(39923,)
(34355,)
P10C32
(39923,)
(29830,)
P10C33
(39923,)
(32232,)
P10C34
(39923,)
(16456,)
P10C35
(39923,)
(24845,)
P10C36
(39923,)
(16961,)
P11C1
(39923,)
(27874,)
P11C2
(39923,)
(18256,)
P11C

P19C1
(39923,)
(27876,)
P19C2
(39923,)
(18244,)
P19C3
(39923,)
(15169,)
P19C4
(39923,)
(23164,)
P19C5
(39923,)
(26862,)
P19C6
(39923,)
(14396,)
P19C7
(39923,)
(24811,)
P19C8
(39923,)
(18327,)
P19C9
(39923,)
(19006,)
P19C10
(39923,)
(24773,)
ERROR2
P19C11
(39923,)
(16444,)
P19C12
(39923,)
(24683,)
P19C13
(39923,)
(14135,)
P19C14
(39923,)
(14406,)
P19C15
(39923,)
(14143,)
P19C16
(39923,)
(16410,)
P19C17
(39923,)
(21933,)
P19C18
(39923,)
(20804,)
P19C19
(39923,)
(29043,)
P19C20
(39923,)
(16461,)
P19C21
(39923,)
(23684,)
P19C22
(39923,)
(24316,)
P19C23
(39923,)
(18515,)
P19C24
(39923,)
(24341,)
P19C25
(39923,)
(31568,)
P19C26
(39923,)
(21579,)
P19C27
(39923,)
(19289,)
P19C28
(39923,)
(23695,)
P19C29
(39923,)
(23380,)
P19C30
(39923,)
(21508,)
P19C31
(39923,)
(34277,)
P19C32
(39923,)
(29793,)
P19C33
(39923,)
(32254,)
P19C34
(39923,)
(16439,)
P19C35
(39923,)
(24826,)
P19C36
(39923,)
(16963,)
P20C1
(39923,)
(27851,)
P20C2
(39923,)
(18251,)
P20C3
(39923,)
(15164,)
P20C4
(39923,)
(23176,)
P20C5


P28C2
(39923,)
(18354,)
P28C3
(39923,)
(15159,)
P28C4
(39923,)
(23164,)
P28C5
(39923,)
(26963,)
P28C6
(39923,)
(14390,)
P28C7
(39923,)
(24850,)
P28C8
(39923,)
(18390,)
P28C9
(39923,)
(19077,)
ERROR2
P28C10
(39923,)
(24752,)
P28C11
(39923,)
(16563,)
P28C12
(39923,)
(24776,)
P28C13
(39923,)
(14259,)
P28C14
(39923,)
(14386,)
P28C15
(39923,)
(14123,)
P28C16
(39923,)
(16439,)
P28C17
(39923,)
(22052,)
P28C18
(39923,)
(20823,)
P28C19
(39923,)
(29047,)
P28C20
(39923,)
(16579,)
P28C21
(39923,)
(23754,)
P28C22
(39923,)
(24376,)
P28C23
(39923,)
(18488,)
P28C24
(39923,)
(24435,)
ERROR2
P28C25
(39923,)
(31656,)
P28C26
(39923,)
(21682,)
P28C27
(39923,)
(19277,)
P28C28
(39923,)
(23728,)
P28C29
(39923,)
(23523,)
P28C30
(39923,)
(21480,)
P28C31
(39923,)
(34431,)
P28C32
(39923,)
(29965,)
P28C33
(39923,)
(32351,)
ERROR2
P28C34
(39923,)
(16562,)
P28C35
(39923,)
(24979,)
P28C36
(39923,)
(16948,)
P29C1
(39923,)
(27826,)
P29C2
(39923,)
(18320,)
P29C3
(39923,)
(15162,)
P29C4
(39923,)
(23161,)
ERROR2
P29C5
(39

P37C5
(39923,)
(26776,)
P37C6
(39923,)
(14408,)
P37C7
(39923,)
(24511,)
P37C8
(39923,)
(18263,)
P37C9
(39923,)
(19044,)
P37C10
(39923,)
(24519,)
P37C11
(39923,)
(16479,)
P37C12
(39923,)
(24409,)
P37C13
(39923,)
(14172,)
P37C14
(39923,)
(14542,)
P37C15
(39923,)
(14159,)
P37C16
(39923,)
(16166,)
P37C17
(39923,)
(22044,)
P37C18
(39923,)
(20865,)
P37C19
(39923,)
(28956,)
P37C20
(39923,)
(16478,)
P37C21
(39923,)
(23641,)
ERROR2
P37C22
(39923,)
(24210,)
P37C23
(39923,)
(18533,)
P37C24
(39923,)
(24357,)
P37C25
(39923,)
(38573,)
P37C26
(39923,)
(21377,)
P37C27
(39923,)
(19383,)
P37C28
(39923,)
(23772,)
P37C29
(39923,)
(23412,)
P37C30
(39923,)
(21215,)
P37C31
(39923,)
(33886,)
P37C32
(39923,)
(29443,)
P37C33
(39923,)
(31833,)
P37C34
(39923,)
(16480,)
ERROR2
P37C35
(39923,)
(24562,)
P37C36
(39923,)
(17006,)
P38C1
(39923,)
(27524,)
ERROR2
P38C2
(39923,)
(18273,)
P38C3
(39923,)
(15194,)
P38C4
(39923,)
(22903,)
P38C5
(39923,)
(28899,)
ERROR2
P38C6
(39923,)
(14542,)
P38C7
(39923,)
(24501,)
P38C8
(39

P46C1
(39923,)
(27440,)
P46C2
(39923,)
(18290,)
P46C3
(39923,)
(15197,)
P46C4
(39923,)
(22924,)
P46C5
(39923,)
(26849,)
P46C6
(39923,)
(14435,)
P46C7
(39923,)
(24610,)
P46C8
(39923,)
(18283,)
P46C9
(39923,)
(19054,)
ERROR2
P46C10
(39923,)
(24546,)
P46C11
(39923,)
(16481,)
P46C12
(39923,)
(24420,)
P46C13
(39923,)
(14255,)
P46C14
(39923,)
(14438,)
P46C15
(39923,)
(14283,)
P46C16
(39923,)
(16188,)
P46C17
(39923,)
(21697,)
ERROR2
P46C18
(39923,)
(21020,)
P46C19
(39923,)
(28986,)
ERROR2
P46C20
(39923,)
(16483,)
P46C21
(39923,)
(23507,)
P46C22
(39923,)
(24286,)
P46C23
(39923,)
(18551,)
P46C24
(39923,)
(24360,)
ERROR2
P46C25
(39923,)
(31319,)
P46C26
(39923,)
(21373,)
P46C27
(39923,)
(19317,)
P46C28
(39923,)
(23698,)
P46C29
(39923,)
(23583,)
P46C30
(39923,)
(21219,)
P46C31
(39923,)
(34015,)
ERROR2
P46C32
(39923,)
(29545,)
P46C33
(39923,)
(31865,)
P46C34
(39923,)
(16479,)
P46C35
(39923,)
(24663,)
P46C36
(39923,)
(16993,)
P47C1
(39923,)
(27467,)
P47C2
(39923,)
(18440,)
P47C3
(39923,)
(15359,)
P4

P55C1
(39923,)
(27482,)
ERROR2
P55C2
(39923,)
(18276,)
P55C3
(39923,)
(15184,)
P55C4
(39923,)
(22901,)
P55C5
(39923,)
(26868,)
P55C6
(39923,)
(14414,)
P55C7
(39923,)
(24432,)
P55C8
(39923,)
(18393,)
P55C9
(39923,)
(19237,)
P55C10
(39923,)
(24468,)
P55C11
(39923,)
(16469,)
P55C12
(39923,)
(24373,)
P55C13
(39923,)
(14164,)
P55C14
(39923,)
(14436,)
P55C15
(39923,)
(14290,)
P55C16
(39923,)
(16203,)
P55C17
(39923,)
(21678,)
P55C18
(39923,)
(21002,)
P55C19
(39923,)
(28924,)
P55C20
(39923,)
(16477,)
P55C21
(39923,)
(23482,)
P55C22
(39923,)
(24205,)
P55C23
(39923,)
(18524,)
P55C24
(39923,)
(24385,)
P55C25
(39923,)
(31425,)
P55C26
(39923,)
(21375,)
P55C27
(39923,)
(19311,)
P55C28
(39923,)
(23694,)
P55C29
(39923,)
(23420,)
P55C30
(39923,)
(21301,)
P55C31
(39923,)
(33935,)
P55C32
(39923,)
(29453,)
ERROR2
P55C33
(39923,)
(31824,)
P55C34
(39923,)
(16471,)
P55C35
(39923,)
(24553,)
P55C36
(39923,)
(17006,)
ERROR2
P56C1
(39923,)
(29601,)
ERROR2
P56C2
(39923,)
(18280,)
P56C3
(39923,)
(15190,)
P56C4
(39

In [131]:
def calculateBPMLib(seriesIn, colName):
    print(colName)
    fs = 256 #The example dataset was recorded at 256Hz
    
    series = seriesIn.copy()
    print(series.shape)
    dropIdx = len(series.dropna())
    series = series.truncate(after=dropIdx-1)
    print(series.shape)

    minuteHz = 7680
    currIdx = 7680
    bpmList = []
    
#     if(len(series)<minuteHz):
#         print('ERROR0')
#         return []
    
    while currIdx < len(series):
#         print(currIdx)
        currSlice = series.iloc[currIdx-minuteHz:currIdx]
        currIdx += 256
        currSlice = np.array(currSlice.to_list())
        try:
            bpmList.append(hp.process(currSlice, fs)[1]['bpm'])
        except hp.exceptions.BadSignalWarning:
            bpmList.append(np.nan)

    return bpmList

In [132]:
hrDf = pd.DataFrame()

for col in df.columns:
    hrDf = pd.concat([hrDf, pd.DataFrame({col : calculateBPMLib(dfRaw[col], col)})], axis=1)

P01C1
(39923,)
(27946,)
P01C2
(39923,)
(18245,)
P01C3
(39923,)
(15159,)
P01C4
(39923,)
(23250,)
P01C5
(39923,)
(26964,)
P01C6
(39923,)
(14390,)
P01C7
(39923,)
(24829,)
P01C8
(39923,)
(18336,)
P01C9
(39923,)
(19035,)
P01C10
(39923,)
(24765,)
P01C11
(39923,)
(16452,)
P01C12
(39923,)
(24682,)
P01C13
(39923,)
(14137,)
P01C14
(39923,)
(14407,)
P01C15
(39923,)
(14134,)
P01C16
(39923,)
(16475,)
P01C17
(39923,)
(21913,)


C:\Users\travi\anaconda3\envs\moodSwingEnv\lib\site-packages\scipy\interpolate\fitpack2.py:253: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


P01C18
(39923,)
(20824,)
P01C19
(39923,)
(29074,)
P01C20
(39923,)
(16512,)
P01C21
(39923,)
(23739,)
P01C22
(39923,)
(24249,)
P01C23
(39923,)
(18506,)
P01C24
(39923,)
(24409,)
P01C25
(39923,)
(31633,)
P01C26
(39923,)
(21593,)
P01C27
(39923,)
(19267,)
P01C28
(39923,)
(23688,)
P01C29
(39923,)
(23385,)
P01C30
(39923,)
(21571,)
P01C31
(39923,)
(34366,)
P01C32
(39923,)
(29931,)
P01C33
(39923,)
(32364,)
P01C34
(39923,)
(16456,)
P01C35
(39923,)
(24859,)
P01C36
(39923,)
(16957,)
P02C1
(39923,)
(28047,)
P02C2
(39923,)
(18255,)
P02C3
(39923,)
(15164,)
P02C4
(39923,)
(23175,)
P02C5
(39923,)
(26934,)
P02C6
(39923,)
(14395,)
P02C7
(39923,)
(24858,)
P02C8
(39923,)
(18365,)
P02C9
(39923,)
(19010,)
P02C10
(39923,)
(24795,)
P02C11
(39923,)
(16475,)
P02C12
(39923,)
(24717,)
P02C13
(39923,)
(14134,)
P02C14
(39923,)
(14412,)
P02C15
(39923,)
(14142,)
P02C16
(39923,)
(16529,)
P02C17
(39923,)
(21935,)
P02C18
(39923,)
(20810,)
P02C19
(39923,)
(29074,)
P02C20
(39923,)
(16498,)
P02C21
(39923,)
(23689,)
P02C22
(3

C:\Users\travi\anaconda3\envs\moodSwingEnv\lib\site-packages\numpy\ma\core.py:5243: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
C:\Users\travi\anaconda3\envs\moodSwingEnv\lib\site-packages\numpy\core\fromnumeric.py:3621: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


P03C3
(39923,)
(15166,)
P03C4
(39923,)
(23182,)
P03C5
(39923,)
(26915,)
P03C6
(39923,)
(14415,)
P03C7
(39923,)
(24849,)
P03C8
(39923,)
(18356,)
P03C9
(39923,)
(19037,)
P03C10
(39923,)
(24812,)
P03C11
(39923,)
(16475,)
P03C12
(39923,)
(24787,)


...

P21C29
(39923,)
(23380,)
P21C30
(39923,)
(21532,)
P21C31
(39923,)
(34205,)
P21C32
(39923,)
(29819,)
P21C33
(39923,)
(32290,)
P21C34
(39923,)
(16456,)
P21C35
(39923,)
(24854,)
P21C36
(39923,)
(16958,)
P22C1
(39923,)
(27882,)
P22C2
(39923,)
(18267,)
P22C3
(39923,)
(15165,)
P22C4
(39923,)
(23146,)
P22C5
(39923,)
(26884,)
P22C6
(39923,)
(14393,)
P22C7
(39923,)
(24849,)
P22C8
(39923,)
(18341,)
P22C9
(39923,)
(19013,)
P22C10
(39923,)
(24721,)
P22C11
(39923,)
(16449,)
P22C12
(39923,)
(24718,)
P22C13
(39923,)
(14200,)
P22C14
(39923,)
(14393,)
P22C15
(39923,)
(14138,)
P22C16
(39923,)
(16450,)
P22C17
(39923,)
(21945,)
P22C18
(39923,)
(20812,)
P22C19
(39923,)
(29067,)
P22C20
(39923,)
(16459,)
P22C21
(39923,)
(23744,)
P22C22
(39923,)
(24252,)
P22C23
(39923,)
(18508,)
P22C24
(39923,)
(24349,)
P22C25
(39923,)
(31612,)
P22C26
(39923,)
(21602,)
P22C27
(39923,)
(19281,)
P22C28
(39923,)
(23702,)
P22C29
(39923,)
(23396,)
P22C30
(39923,)
(21625,)
P22C31
(39923,)
(34314,)
P22C32
(39923,)
(29792,)
P22C33
(3

P30C36
(39923,)
(17093,)
P31C1
(39923,)
(27934,)
P31C2
(39923,)
(18257,)
P31C3
(39923,)
(15294,)
P31C4
(39923,)
(23361,)
P31C5
(39923,)
(27003,)
P31C6
(39923,)
(14405,)
P31C7
(39923,)
(24921,)
P31C8
(39923,)
(18407,)
P31C9
(39923,)
(19134,)
P31C10
(39923,)
(24638,)
P31C11
(39923,)
(16470,)
P31C12
(39923,)
(24789,)
P31C13
(39923,)
(14226,)
P31C14
(39923,)
(14403,)
P31C15
(39923,)
(14144,)
P31C16
(39923,)
(16472,)
P31C17
(39923,)
(22014,)
P31C18
(39923,)
(20859,)
P31C19
(39923,)
(29221,)
P31C20
(39923,)
(16460,)
P31C21
(39923,)
(23689,)
P31C22
(39923,)
(24273,)
P31C23
(39923,)
(18605,)
P31C24
(39923,)
(24358,)
P31C25
(39923,)
(31812,)
P31C26
(39923,)
(21590,)
P31C27
(39923,)
(19274,)
P31C28
(39923,)
(23730,)
P31C29
(39923,)
(23461,)
P31C30
(39923,)
(21592,)
P31C31
(39923,)
(34436,)
P31C32
(39923,)
(29903,)
P31C33
(39923,)
(32401,)
P31C34
(39923,)
(16485,)
P31C35
(39923,)
(24937,)
P31C36
(39923,)
(16973,)
P32C1
(39923,)
(27975,)
P32C2
(39923,)
(18274,)
P32C3
(39923,)
(15176,)
P32C4
(39923

P40C8
(39923,)
(18275,)
P40C9
(39923,)
(19052,)
P40C10
(39923,)
(24623,)
P40C11
(39923,)
(16469,)
P40C12
(39923,)
(24408,)
P40C13
(39923,)
(14154,)
P40C14
(39923,)
(14413,)
P40C15
(39923,)
(14151,)
P40C16
(39923,)
(16202,)
P40C17
(39923,)
(21695,)
P40C18
(39923,)
(20990,)
P40C19
(39923,)
(28903,)
P40C20
(39923,)
(16475,)
P40C21
(39923,)
(23414,)
P40C22
(39923,)
(24192,)
P40C23
(39923,)
(18522,)
P40C24
(39923,)
(24330,)
P40C25
(39923,)
(31584,)
P40C26
(39923,)
(21350,)
P40C27
(39923,)
(19303,)
P40C28
(39923,)
(23695,)
P40C29
(39923,)
(23427,)
P40C30
(39923,)
(21268,)
P40C31
(39923,)
(33924,)
P40C32
(39923,)
(29475,)
P40C33
(39923,)
(31799,)
P40C34
(39923,)
(16473,)
P40C35
(39923,)
(24640,)
P40C36
(39923,)
(16978,)
P41C1
(39923,)
(27428,)
P41C2
(39923,)
(18398,)
P41C3
(39923,)
(15180,)
P41C4
(39923,)
(22904,)
P41C5
(39923,)
(26784,)
P41C6
(39923,)
(14407,)
P41C7
(39923,)
(24502,)
P41C8
(39923,)
(18277,)
P41C9
(39923,)
(19048,)
P41C10
(39923,)
(24441,)
P41C11
(39923,)
(16478,)
P41C12
(399

P49C16
(39923,)
(16106,)
P49C17
(39923,)
(21686,)
P49C18
(39923,)
(20985,)
P49C19
(39923,)
(28908,)
P49C20
(39923,)
(16605,)
P49C21
(39923,)
(23424,)
P49C22
(39923,)
(24231,)
P49C23
(39923,)
(18532,)
P49C24
(39923,)
(24411,)
P49C25
(39923,)
(31599,)
P49C26
(39923,)
(26711,)
P49C27
(39923,)
(19321,)
P49C28
(39923,)
(23685,)
P49C29
(39923,)
(23434,)
P49C30
(39923,)
(21278,)
P49C31
(39923,)
(34013,)
P49C32
(39923,)
(29412,)
P49C33
(39923,)
(31776,)
P49C34
(39923,)
(16472,)
P49C35
(39923,)
(24562,)
P49C36
(39923,)
(17231,)
P50C1
(39923,)
(27407,)
P50C2
(39923,)
(18285,)
P50C3
(39923,)
(15188,)
P50C4
(39923,)
(23145,)
P50C5
(39923,)
(26870,)
P50C6
(39923,)
(14480,)
P50C7
(39923,)
(24478,)
P50C8
(39923,)
(18273,)
P50C9
(39923,)
(19050,)
P50C10
(39923,)
(24511,)
P50C11
(39923,)
(16475,)
P50C12
(39923,)
(24451,)
P50C13
(39923,)
(14154,)
P50C14
(39923,)
(14422,)
P50C15
(39923,)
(14156,)
P50C16
(39923,)
(16130,)
P50C17
(39923,)
(21652,)
P50C18
(39923,)
(20901,)
P50C19
(39923,)
(29088,)
P50C20
(3

P58C23
(39923,)
(18533,)
P58C24
(39923,)
(24327,)
P58C25
(39923,)
(31471,)
P58C26
(39923,)
(21363,)
P58C27
(39923,)
(19311,)
P58C28
(39923,)
(23677,)
P58C29
(39923,)
(23427,)
P58C30
(39923,)
(21271,)
P58C31
(39923,)
(33994,)
P58C32
(39923,)
(29511,)
P58C33
(39923,)
(31832,)
P58C34
(39923,)
(16488,)
P58C35
(39923,)
(24528,)
P58C36
(39923,)
(17176,)


In [133]:
hrDf.head()

,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
0,76.739416,78.251776,81.153791,75.344028,77.042132,71.597961,78.514931,78.128179,72.560400,80.348736,...,NaN,NaN,NaN,NaN,571.155378,NaN,NaN,NaN,NaN,NaN
1,76.335796,78.084281,80.877578,75.038676,76.051013,71.431478,78.262269,77.713661,72.540818,79.500268,...,NaN,NaN,NaN,NaN,571.155378,NaN,NaN,NaN,NaN,NaN
2,76.141479,77.689338,81.538020,74.987795,76.161587,71.193511,78.063394,77.290902,72.462596,79.202475,...,NaN,NaN,NaN,NaN,571.155378,NaN,NaN,NaN,NaN,NaN
3,76.018697,77.710025,79.857398,75.150856,75.260579,71.101706,78.042519,76.693481,72.619208,79.098809,...,NaN,NaN,NaN,NaN,714.418605,NaN,NaN,NaN,NaN,NaN
4,75.242890,77.628739,79.357061,75.124917,75.580952,71.026556,78.042519,76.603316,72.766649,79.264635,...,NaN,NaN,NaN,NaN,393.846154,NaN,NaN,NaN,NaN,NaN


In [159]:
temp = hrDf.isna().sum()

In [160]:
type(temp)
count = 0

In [161]:
for i in temp.iteritems():
    if i[1] == 126:
        hrDf = hrDf.drop([i[0]], axis=1)
        count+=1


In [162]:
hrDf.shape

(126, 833)

In [163]:
count

0

In [164]:
hrDf.to_csv('data/ASCERTAIN_Good_HR.csv')

In [139]:
 hrDf.isna().sum()

P01C1      46
P01C2      84
P01C3      96
P01C4      65
P01C5      50
         ... 
P58C32    126
P58C33    126
P58C34    126
P58C35    126
P58C36    126
Length: 2088, dtype: int64

In [127]:
retData = hp.process(dataList, sample_rate=256)

In [130]:
retData[1]['bpm']

74.0925576671986

In [134]:
hrDf.to_csv('data/ASCERTAIN_HR_Processed.csv', index=False)

In [37]:
hrDf.shape

(96, 2088)

In [37]:
df = pd.read_csv('data/ASCERTAIN_HR_Processed.csv')

In [41]:
df.head(10)

,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
0,76.0,79.0,NaN,75.0,74.0,NaN,78.0,79.0,73.0,75.0,...,275.0,NaN,NaN,101.0,NaN,NaN,NaN,675.0,308.0,69.0
1,77.0,79.0,NaN,75.0,76.0,NaN,78.0,78.0,73.0,76.0,...,268.0,NaN,NaN,93.0,NaN,NaN,NaN,519.0,1911.0,63.0
2,77.0,78.0,NaN,75.0,84.0,NaN,78.0,78.0,73.0,76.0,...,258.0,NaN,NaN,91.0,NaN,NaN,NaN,519.0,1911.0,202.0
3,78.0,78.0,NaN,75.0,75.0,NaN,78.0,77.0,73.0,76.0,...,273.0,NaN,NaN,111.0,NaN,NaN,NaN,475.0,1911.0,253.0
4,77.0,78.0,NaN,75.0,75.0,NaN,78.0,76.0,74.0,78.0,...,263.0,NaN,NaN,131.0,NaN,NaN,NaN,475.0,1911.0,279.0
5,77.0,78.0,NaN,75.0,75.0,NaN,78.0,76.0,74.0,79.0,...,262.0,NaN,NaN,142.0,NaN,NaN,NaN,NaN,1463.0,284.0
6,78.0,78.0,NaN,75.0,75.0,NaN,78.0,76.0,74.0,79.0,...,278.0,NaN,NaN,183.0,NaN,NaN,NaN,NaN,149.0,275.0
7,78.0,78.0,NaN,74.0,75.0,NaN,78.0,76.0,74.0,79.0,...,282.0,NaN,NaN,185.0,NaN,NaN,NaN,NaN,258.0,277.0
8,77.0,78.0,NaN,74.0,76.0,NaN,78.0,76.0,74.0,79.0,...,275.0,NaN,NaN,187.0,NaN,NaN,NaN,NaN,261.0,NaN
9,76.0,78.0,NaN,75.0,75.0,NaN,78.0,76.0,74.0,79.0,...,277.0,NaN,NaN,191.0,NaN,NaN,NaN,NaN,259.0,NaN


In [44]:
lenList = []
for col in df.columns:
    lenList.append(len(df[col].dropna()))

In [48]:
np.sum(lenList)/len(lenList)

23.26963601532567

# Label Data

In [7]:
labels = pd.read_csv('data/ASCERTAIN_Emotions_raw.csv', index_col='RowIdx')

In [8]:
labels.head()

,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
RowIdx,,,,,,,,,,,,,,,,,,,,,
Arousal,1,3,1,6,4,4,3,4,4,4,...,3,5,4,4,4,4,4,4,4,4
Valence,1,2,-2,0,1,0,1,2,2,3,...,-2,-2,-2,-1,-1,-1,-1,-3,-2,-2
Engagement,4,5,3,5,4,4,5,5,5,5,...,5,5,5,5,5,5,5,4,5,5
Liking,5,4,3,3,4,3,4,5,5,6,...,1,1,1,2,2,1,2,0,1,1
Familiarity,2,0,0,0,1,0,3,0,0,4,...,1,0,0,0,3,2,1,0,2,3


In [13]:
labels.sum().sum()/labels.shape[1]

13.69492337164751

In [14]:
labelsTop3 = labels.drop(['Familiarity', 'Liking'], axis=0)

In [15]:
labelsTop3.sum().sum()/labelsTop3.shape[1]

8.692528735632184

In [16]:
labelsTop3.head()

,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
RowIdx,,,,,,,,,,,,,,,,,,,,,
Arousal,1,3,1,6,4,4,3,4,4,4,...,3,5,4,4,4,4,4,4,4,4
Valence,1,2,-2,0,1,0,1,2,2,3,...,-2,-2,-2,-1,-1,-1,-1,-3,-2,-2
Engagement,4,5,3,5,4,4,5,5,5,5,...,5,5,5,5,5,5,5,4,5,5


In [18]:
labelsTop3.sum().max(), labelsTop3.sum().min()

(15.0, -3.0)

In [19]:
listData = []
for i in labelsTop3.sum():
    if i >8.5:
        listData.append('1')
    else:
        listData.append('0')

In [21]:
labelsTop3.sum()

P01C1      6.0
P01C2     10.0
P01C3      2.0
P01C4     11.0
P01C5      9.0
          ... 
P58C32     8.0
P58C33     8.0
P58C34     5.0
P58C35     7.0
P58C36     7.0
Length: 2088, dtype: float64

In [31]:
outLabels = pd.DataFrame(listData,index=labelsTop3.columns).T

In [32]:
outLabels.head()

,P01C1,P01C2,P01C3,P01C4,P01C5,P01C6,P01C7,P01C8,P01C9,P01C10,...,P58C27,P58C28,P58C29,P58C30,P58C31,P58C32,P58C33,P58C34,P58C35,P58C36
0,0,1,0,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
outLabels.to_csv('data/ASCERTAIN_labels.csv')

In [34]:
outLabelsComplicated = labelsTop3.drop(['Engagement'])

In [36]:
outLabelsComplicated.to_csv('data/ASCERTAIN_complicatedLabels.csv')

In [167]:
for col in labelsTop3.columns:
    if col not in hrDf.columns:
        labelsTop3 = labelsTop3.drop([col], axis=1)

In [168]:
labelsTop3.shape

(3, 833)

In [169]:
labelsTop3.to_csv('data/ASCERTAIN_labels_cleaned.csv')